## Goede Voornemens het hele jaar door
Bespreking API backend - Python AWS Lambda CloudFunctions. 

In [1]:
import json
import requests

cloudfunction = 'https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw'

req = {
  "body": {
   }
}

vdata = requests.post(cloudfunction, data=json.dumps(req))

vdata.json()


[{'username': 'Joep!', 'wwdwid': 'wwdwid2022010780bh'},
 {'username': 'Sander!', 'wwdwid': 'wwdwid202201072841'}]

## Bespreking volgt... 7-jan 16:00